## **Import Libraries**

In [17]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder
from sklearn.impute import SimpleImputer

## **Import Data**

In [18]:
df=pd.read_csv("./data/covid_toy.csv")
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [19]:
df['cough'].value_counts()

cough
Mild      62
Strong    38
Name: count, dtype: int64

In [20]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

## **1.Simple Method**

In [21]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df.drop('has_covid',axis=1),df['has_covid'],
                                                       test_size=0.3,random_state=42)

In [22]:
# Let's deal with fever column of 
si=SimpleImputer()
X_train_fever=si.fit_transform(X_train[['fever']])
X_test_fever=si.transform(X_test[['fever']])
X_train_fever.shape

(70, 1)

In [23]:
# Let's deal with gender and city columns
ohe=OneHotEncoder(sparse_output=False,drop='first')
X_train_gender_city=ohe.fit_transform(X_train[['gender','city']])
X_test_gender_city=ohe.fit_transform(X_test[['gender','city']])
X_train_gender_city.shape

(70, 4)

In [24]:
# Let's deal with cough column
oe=OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough=oe.fit_transform(X_train[['cough']])
X_test_cough=oe.fit_transform(X_test[['cough']])
X_train_cough.shape

(70, 1)

In [25]:
# Extracting age column
X_train_age=X_train.drop(columns=['gender','fever','cough','city']).values
X_test_age=X_test.drop(columns=['gender','fever','cough','city']).values
X_train_age.shape

(70, 1)

#### **`Transformed Data`**

In [26]:
X_train_transformed=np.concatenate((X_train_age,X_train_gender_city,X_train_fever,X_train_cough),axis=1)
X_test_transformed=np.concatenate((X_test_age,X_test_gender_city,X_test_fever,X_test_cough),axis=1)
X_train_transformed.shape

(70, 7)

## **2.Column Transformer**

In [27]:
from sklearn.compose import ColumnTransformer

In [28]:
transformer=ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong',]]),['cough']),
    ('tnf3',OneHotEncoder(sparse_output=False,drop='first'),['gender','city']),
],remainder='passthrough')

In [29]:
transformer.fit_transform(X_train).shape

(70, 7)

In [30]:
transformer.transform(X_test).shape

(30, 7)